In [1]:
import cv2
from yolo_predictions import YOLO_pred

#yolo = YOLO_pred('./Model/weights/best.onnx','data.yaml')

#img = cv2.imread('./firee.jpg')
#cv2.imshow('img',img)
#cv2.waitKey(0)
#cv2.destroyAllWindows()

#img_pred = yolo.predictions(img)

#cv2.imshow('prediction image', img_pred)
#cv2.waitKey(0)
#cv2.destroyAllWindows()

In [2]:
yolo = YOLO_pred('./Model/weights/best.onnx','data.yaml')

In [3]:
img = cv2.imread('./ffff.jpg')
cv2.imshow('img',img)
cv2.waitKey(0)
cv2.destroyAllWindows()

error: OpenCV(4.9.0) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window.cpp:971: error: (-215:Assertion failed) size.width>0 && size.height>0 in function 'cv::imshow'


In [18]:
img_pred = yolo.predictions(img)

AttributeError: 'tuple' object has no attribute 'flatten'

In [4]:
cv2.imshow('prediction image', img_pred)
cv2.waitKey(0)
cv2.destroyAllWindows()

NameError: name 'img_pred' is not defined

In [17]:
import cv2
import numpy as np
import os
import yaml
from yaml.loader import SafeLoader

class YOLO_pred():
    def __init__(self, onnx_model, data_yaml):
        # load YAML 
        with open('data.yaml', mode='r') as f:
            data_yaml = yaml.load(f, Loader=SafeLoader)
          
        self.labels = data_yaml['names']
        self.nc = data_yaml['nc']
    
        # load yolo model
        self.yolo = cv2.dnn.readNetFromONNX('./Model/weights/best.onnx')
        self.yolo.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
        self.yolo.setPreferableTarget(cv2.dnn.DNN_TARGET_CPU)

    def predictions(self, frame):   
        # get yolo prediction from the frame
        # step 1: convert frame into square img (array)
        row, col, d = frame.shape
        max_rc = max(row, col)
        input_image = np.zeros((max_rc, max_rc, 3), dtype=np.uint8)
        input_image[0:row, 0:col] = frame 

        # step 2: get prediction from square array
        INPUT_WH_YOLO = 640
        blob = cv2.dnn.blobFromImage(input_image, 1/255, (INPUT_WH_YOLO, INPUT_WH_YOLO), swapRB=True, crop=False)
        self.yolo.setInput(blob)
        preds = self.yolo.forward()  # detection or prediction from yolo 

        detections = preds[0]
        boxes = []
        confidences = []
        classes = []
        # width and height of the frame(input_image)
        image_w, image_h = input_image.shape[:2]
        x_factor = image_w / INPUT_WH_YOLO
        y_factor = image_h / INPUT_WH_YOLO

        for i in range(len(detections)):
            row = detections[i]
            confidence = row[4]  # confidence of detecting an object
            if confidence > 0.4:
                class_score = row[5:].max()  # maximum probability of the object 
                class_id = row[5:].argmax()  # get the index position at which max probability occur
                if class_score > 0.25:
                    cx, cy, w, h = row[0:4]
                    # construct bounding box from four values
                    # left top width height
                    left = int((cx - 0.5*w) * x_factor)
                    top = int((cy - 0.5*h) * y_factor)
                    width = int(w * x_factor)
                    height = int(h * y_factor)
            
                    box = np.array([left, top, width, height])
            
                    # append values into the list
                    confidences.append(confidence)
                    boxes.append(box)
                    classes.append(class_id)

        # Clean the data
        boxes_np = np.array(boxes)  # Convert to numpy array
        confidences_np = np.array(confidences)

        # Apply Non-Maximum Suppression (NMS)
        index = cv2.dnn.NMSBoxes(boxes_np.tolist(), confidences_np.tolist(), 0.25, 0.45).flatten()

        # Draw bounding boxes and labels on the frame
        for ind in index:
            # extract bounding boxes
            x, y, w, h = boxes_np[ind]
            bb_conf = int(confidences_np[ind] * 100)
            classes_id = classes[ind]
            class_name = self.labels[classes_id]
            colors = self.generate_colors(classes_id)

            text = f'{class_name}:{bb_conf}%'

            cv2.rectangle(frame, (x, y), (x+w, y+h), colors, 2) 
            cv2.rectangle(frame, (x, y-30), (x+w, y), colors, -1)
            cv2.putText(frame, text, (x, y-10), cv2.FONT_HERSHEY_PLAIN, 0.7, (0, 0, 0), 1)

        return frame

    def generate_colors(self, ID):
        np.random.seed(10)
        colors = np.random.randint(100, 255, size=(self.nc, 3)).tolist()
        return tuple(colors[ID])


In [16]:
import cv2
import numpy as np
import yaml
from yaml.loader import SafeLoader

class YOLO_pred:
    def __init__(self, onnx_model, data_yaml):
        # Load YAML
        with open(data_yaml, mode='r') as f:
            data_yaml = yaml.load(f, Loader=SafeLoader)

        self.labels = data_yaml['names']
        self.nc = data_yaml['nc']

        # Load YOLO model
        self.yolo = cv2.dnn.readNetFromONNX(onnx_model)
        self.yolo.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
        self.yolo.setPreferableTarget(cv2.dnn.DNN_TARGET_CPU)

    def predictions(self, frame):
        # Get YOLO prediction from the frame
        row, col, d = frame.shape
        max_rc = max(row, col)
        input_image = np.zeros((max_rc, max_rc, 3), dtype=np.uint8)
        input_image[0:row, 0:col] = frame

        # Resize the image to fit YOLO input size
        INPUT_WH_YOLO = 640
        blob = cv2.dnn.blobFromImage(input_image, 1/255, (INPUT_WH_YOLO, INPUT_WH_YOLO), swapRB=True, crop=False)
        self.yolo.setInput(blob)
        preds = self.yolo.forward()  # Detection or prediction from YOLO

        detections = preds[0]
        boxes = []
        confidences = []
        classes = []
        # Width and height of the frame (input_image)
        image_w, image_h = input_image.shape[:2]
        x_factor = image_w / INPUT_WH_YOLO
        y_factor = image_h / INPUT_WH_YOLO

        for i in range(len(detections)):
            row = detections[i]
            confidence = row[4]  # Confidence of detecting an object
            if confidence > 0.4:
                class_score = row[5:].max()  # Maximum probability of the object
                class_id = row[5:].argmax()  # Get the index position at which max probability occurs
                if class_score > 0.25:
                    cx, cy, w, h = row[0:4]
                    # Construct bounding box from four values: left top width height
                    left = int((cx - 0.5*w) * x_factor)
                    top = int((cy - 0.5*h) * y_factor)
                    width = int(w * x_factor)
                    height = int(h * y_factor)

                    box = np.array([left, top, width, height])

                    # Append values into the list
                    confidences.append(confidence)
                    boxes.append(box)
                    classes.append(class_id)

        # Clean the data
        boxes_np = np.array(boxes)
        confidences_np = np.array(confidences)

        # Apply Non-Maximum Suppression (NMS)
        nms_result = cv2.dnn.NMSBoxes(boxes_np.tolist(), confidences_np.flatten().tolist(), 0.25, 0.45)
        for i in range(len(nms_result)):
            ind = nms_result[i][0]

            # Draw bounding boxes and labels on the frame
            x, y, w, h = boxes_np[ind]
            bb_conf = int(confidences_np[ind] * 100)
            classes_id = classes[ind]
            class_name = self.labels[classes_id]
            colors = self.generate_colors(classes_id)

            text = f'{class_name}:{bb_conf}%'

            cv2.rectangle(frame, (x, y), (x+w, y+h), colors, 2)
            cv2.rectangle(frame, (x, y-30), (x+w, y), colors, -1)
            cv2.putText(frame, text, (x, y-10), cv2.FONT_HERSHEY_PLAIN, 0.7, (0, 0, 0), 1)
                        
        return frame

    def generate_colors(self, ID):
        np.random.seed(10)
        colors = np.random.randint(100, 255, size=(self.nc, 3)).tolist()
        return tuple(colors[ID])

def main():
    # Paths to the ONNX model and data YAML
    onnx_model = './Model/weights/best.onnx'
    data_yaml = 'data.yaml'

    yolo_detector = YOLO_pred(onnx_model, data_yaml)

    # Open camera
    cap = cv2.VideoCapture(0)  # 0 for default camera, change it if you have multiple cameras

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Perform predictions on the frame
        frame_with_predictions = yolo_detector.predictions(frame)

        # Display the frame with predictions
        cv2.imshow('YOLO Fire Detection', frame_with_predictions)
        
        # Press 'q' to exit
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Release resources
    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()


In [36]:
#real time code

import cv2
import numpy as np
import yaml
from yaml.loader import SafeLoader

class YOLO_pred:
    def __init__(self, onnx_model, data_yaml):
        # Load YAML
        with open(data_yaml, mode='r') as f:
            data_yaml = yaml.load(f, Loader=SafeLoader)

        self.labels = data_yaml['names']
        self.nc = data_yaml['nc']

        # Load YOLO model
        self.yolo = cv2.dnn.readNetFromONNX(onnx_model)
        self.yolo.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
        self.yolo.setPreferableTarget(cv2.dnn.DNN_TARGET_CPU)

    def predictions(self, frame):
        # Get YOLO prediction from the frame
        row, col, d = frame.shape
        max_rc = max(row, col)
        input_image = np.zeros((max_rc, max_rc, 3), dtype=np.uint8)
        input_image[0:row, 0:col] = frame

        # Resize the image to fit YOLO input size
        INPUT_WH_YOLO = 640
        blob = cv2.dnn.blobFromImage(input_image, 1/255, (INPUT_WH_YOLO, INPUT_WH_YOLO), swapRB=True, crop=False)
        self.yolo.setInput(blob)
        preds = self.yolo.forward()  # Detection or prediction from YOLO

        detections = preds[0]
        boxes = []
        confidences = []
        classes = []
        # Width and height of the frame (input_image)
        image_w, image_h = input_image.shape[:2]
        x_factor = image_w / INPUT_WH_YOLO
        y_factor = image_h / INPUT_WH_YOLO

        for i in range(len(detections)):
            row = detections[i]
            confidence = row[4]  # Confidence of detecting an object
            if confidence > 0.4:
                class_score = row[5:].max()  # Maximum probability of the object
                class_id = row[5:].argmax()  # Get the index position at which max probability occurs
                if class_score > 0.25:
                    cx, cy, w, h = row[0:4]
                    # Construct bounding box from four values: left top width height
                    left = int((cx - 0.5*w) * x_factor)
                    top = int((cy - 0.5*h) * y_factor)
                    width = int(w * x_factor)
                    height = int(h * y_factor)

                    box = np.array([left, top, width, height])

                    # Append values into the list
                    confidences.append(confidence)
                    boxes.append(box)
                    classes.append(class_id)

        # Clean the data
        boxes_np = np.array(boxes)
        confidences_np = np.array(confidences)

        # Apply Non-Maximum Suppression (NMS)
        nms_result = cv2.dnn.NMSBoxes(boxes_np.tolist(), confidences_np.flatten().tolist(), 0.25, 0.45)
        if len(nms_result) > 0:
            index = nms_result.flatten()
            # Draw bounding boxes and labels on the frame
            for ind in index:
                # Extract bounding boxes
                x, y, w, h = boxes_np[ind]
                bb_conf = int(confidences_np[ind] * 100)
                classes_id = classes[ind]
                class_name = self.labels[classes_id]
                colors = self.generate_colors(classes_id)

                text = f'{class_name}:{bb_conf}%'

                cv2.rectangle(frame, (x, y), (x+w, y+h), colors, 2)
                cv2.rectangle(frame, (x, y-30), (x+w, y), colors, -1)
                cv2.putText(frame, text, (x, y-10), cv2.FONT_HERSHEY_PLAIN, 0.7, (0, 0, 0), 1)
                        
        return frame

    def generate_colors(self, ID):
        np.random.seed(10)
        colors = np.random.randint(100, 255, size=(self.nc, 3)).tolist()
        return tuple(colors[ID])

def main():
    # Paths to the ONNX model and data YAML
    onnx_model = './Model/weights/best.onnx'
    data_yaml = 'data.yaml'

    yolo_detector = YOLO_pred(onnx_model, data_yaml)

    # Open camera
    cap = cv2.VideoCapture(0)  # 0 for default camera, change it if you have multiple cameras

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Perform predictions on the frame
        frame_with_predictions = yolo_detector.predictions(frame)

        # Display the frame with predictions
        cv2.imshow('YOLO Fire Detection', frame_with_predictions)
        
        # Press 'q' to exit
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Release resources
    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()


In [33]:
#video code

import cv2
import numpy as np
import yaml
from yaml.loader import SafeLoader

class YOLO_pred:
    def __init__(self, onnx_model, data_yaml):
        # Load YAML
        with open(data_yaml, mode='r') as f:
            data_yaml = yaml.load(f, Loader=SafeLoader)

        self.labels = data_yaml['names']
        self.nc = data_yaml['nc']

        # Load YOLO model
        self.yolo = cv2.dnn.readNetFromONNX(onnx_model)
        self.yolo.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
        self.yolo.setPreferableTarget(cv2.dnn.DNN_TARGET_CPU)

    def predictions(self, frame):
        # Get YOLO prediction from the frame
        row, col, d = frame.shape
        max_rc = max(row, col)
        input_image = np.zeros((max_rc, max_rc, 3), dtype=np.uint8)
        input_image[0:row, 0:col] = frame

        # Resize the image to fit YOLO input size
        INPUT_WH_YOLO = 640
        blob = cv2.dnn.blobFromImage(input_image, 1/255, (INPUT_WH_YOLO, INPUT_WH_YOLO), swapRB=True, crop=False)
        self.yolo.setInput(blob)
        preds = self.yolo.forward()  # Detection or prediction from YOLO

        detections = preds[0]
        boxes = []
        confidences = []
        classes = []
        # Width and height of the frame (input_image)
        image_w, image_h = input_image.shape[:2]
        x_factor = image_w / INPUT_WH_YOLO
        y_factor = image_h / INPUT_WH_YOLO

        for i in range(len(detections)):
            row = detections[i]
            confidence = row[4]  # Confidence of detecting an object
            if confidence > 0.4:
                class_score = row[5:].max()  # Maximum probability of the object
                class_id = row[5:].argmax()  # Get the index position at which max probability occurs
                if class_score > 0.25:
                    cx, cy, w, h = row[0:4]
                    # Construct bounding box from four values: left top width height
                    left = int((cx - 0.5*w) * x_factor)
                    top = int((cy - 0.5*h) * y_factor)
                    width = int(w * x_factor)
                    height = int(h * y_factor)

                    box = np.array([left, top, width, height])

                    # Append values into the list
                    confidences.append(confidence)
                    boxes.append(box)
                    classes.append(class_id)

        # Clean the data
        boxes_np = np.array(boxes)
        confidences_np = np.array(confidences)

        # Apply Non-Maximum Suppression (NMS)
        nms_result = cv2.dnn.NMSBoxes(boxes_np.tolist(), confidences_np.flatten().tolist(), 0.25, 0.45)
        if len(nms_result) > 0:
            index = nms_result.flatten()
            # Draw bounding boxes and labels on the frame
            for ind in index:
                # Extract bounding boxes
                x, y, w, h = boxes_np[ind]
                bb_conf = int(confidences_np[ind] * 100)
                classes_id = classes[ind]
                class_name = self.labels[classes_id]
                colors = self.generate_colors(classes_id)

                text = f'{class_name}:{bb_conf}%'

                cv2.rectangle(frame, (x, y), (x+w, y+h), colors, 2)
                cv2.rectangle(frame, (x, y-30), (x+w, y), colors, -1)
                cv2.putText(frame, text, (x, y-10), cv2.FONT_HERSHEY_PLAIN, 0.7, (0, 0, 0), 1)
                        
        return frame

    def generate_colors(self, ID):
        np.random.seed(10)
        colors = np.random.randint(100, 255, size=(self.nc, 3)).tolist()
        return tuple(colors[ID])

def main():
    # Paths to the ONNX model and data YAML
    onnx_model = './Model/weights/best.onnx'
    data_yaml = 'data.yaml'

    yolo_detector = YOLO_pred(onnx_model, data_yaml)

    # Path to the input video file
    video_file = 'fire.mp4'

    # Open the video file
    cap = cv2.VideoCapture(video_file)

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Perform predictions on the frame
        frame_with_predictions = yolo_detector.predictions(frame)

        # Display the frame with predictions
        cv2.imshow('YOLO Fire Detection', frame_with_predictions)
        
        # Press 'q' to exit
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Release resources
    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()


In [16]:
import cv2
import numpy as np
import yaml
from yaml.loader import SafeLoader

class YOLO_pred:
    def __init__(self, onnx_model, data_yaml):
        # Load YAML
        with open(data_yaml, mode='r') as f:
            data_yaml = yaml.load(f, Loader=SafeLoader)

        self.labels = data_yaml['names']
        self.nc = data_yaml['nc']

        # Load YOLO model
        self.yolo = cv2.dnn.readNetFromONNX(onnx_model)
        self.yolo.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
        self.yolo.setPreferableTarget(cv2.dnn.DNN_TARGET_CPU)

    def predictions(self, frame):
        # Get YOLO prediction from the frame
        row, col, d = frame.shape
        max_rc = max(row, col)
        input_image = np.zeros((max_rc, max_rc, 3), dtype=np.uint8)
        input_image[0:row, 0:col] = frame

        # Resize the image to fit YOLO input size
        INPUT_WH_YOLO = 640
        blob = cv2.dnn.blobFromImage(input_image, 1/255, (INPUT_WH_YOLO, INPUT_WH_YOLO), swapRB=True, crop=False)
        self.yolo.setInput(blob)
        preds = self.yolo.forward()  # Detection or prediction from YOLO

        detections = preds[0]
        boxes = []
        confidences = []
        classes = []
        # Width and height of the frame (input_image)
        image_w, image_h = input_image.shape[:2]
        x_factor = image_w / INPUT_WH_YOLO
        y_factor = image_h / INPUT_WH_YOLO

        for i in range(len(detections)):
            row = detections[i]
            confidence = row[4]  # Confidence of detecting an object
            if confidence > 0.4:
                class_score = row[5:].max()  # Maximum probability of the object
                class_id = row[5:].argmax()  # Get the index position at which max probability occurs
                if class_score > 0.25:
                    cx, cy, w, h = row[0:4]
                    # Construct bounding box from four values: left top width height
                    left = int((cx - 0.5*w) * x_factor)
                    top = int((cy - 0.5*h) * y_factor)
                    width = int(w * x_factor)
                    height = int(h * y_factor)

                    box = np.array([left, top, width, height])

                    # Append values into the list
                    confidences.append(confidence)
                    boxes.append(box)
                    classes.append(class_id)

        # Clean the data
        boxes_np = np.array(boxes)
        confidences_np = np.array(confidences)

        # Apply Non-Maximum Suppression (NMS)
        nms_result = cv2.dnn.NMSBoxes(boxes_np.tolist(), confidences_np.flatten().tolist(), 0.25, 0.45)
        if len(nms_result) > 0:
            index = nms_result.flatten()
            # Draw bounding boxes and labels on the frame
            for ind in index:
                # Extract bounding boxes
                x, y, w, h = boxes_np[ind]
                bb_conf = int(confidences_np[ind] * 100)
                classes_id = classes[ind]
                class_name = self.labels[classes_id]
                colors = self.generate_colors(classes_id)

                text = f'{class_name}:{bb_conf}%'

                cv2.rectangle(frame, (x, y), (x+w, y+h), colors, 2)
                cv2.rectangle(frame, (x, y-30), (x+w, y), colors, -1)
                cv2.putText(frame, text, (x, y-10), cv2.FONT_HERSHEY_PLAIN, 0.7, (0, 0, 0), 1)
                        
        return frame

    def generate_colors(self, ID):
        np.random.seed(10)
        colors = np.random.randint(100, 255, size=(self.nc, 3)).tolist()
        return tuple(colors[ID])

def main():
    # Paths to the ONNX model and data YAML
    onnx_model = './Model/weights/best.onnx'
    data_yaml = 'data.yaml'

    yolo_detector = YOLO_pred(onnx_model, data_yaml)

    # Path to the input video file
    video_file = 'fire.mp4'

    # Open the video file
    cap = cv2.VideoCapture(video_file)

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Perform predictions on the frame
        frame_with_predictions = yolo_detector.predictions(frame)

        # Display the frame with predictions
        cv2.imshow('YOLO Fire Detection', frame_with_predictions)
        
        # Press 'q' to exit
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Release resources
    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()


In [15]:
import cv2
import numpy as np
import yaml
from yaml.loader import SafeLoader

class YOLO_pred:
    def __init__(self, onnx_model, data_yaml):
        # Load YAML
        with open(data_yaml, mode='r') as f:
            data_yaml = yaml.load(f, Loader=SafeLoader)

        self.labels = data_yaml['names']
        self.nc = data_yaml['nc']

        # Load YOLO model
        self.yolo = cv2.dnn.readNetFromONNX(onnx_model)
        self.yolo.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
        self.yolo.setPreferableTarget(cv2.dnn.DNN_TARGET_CPU)

    def predictions(self, frame):
        # Get YOLO prediction from the frame
        row, col, d = frame.shape
        max_rc = max(row, col)
        input_image = np.zeros((max_rc, max_rc, 3), dtype=np.uint8)
        input_image[0:row, 0:col] = frame

        # Resize the image to fit YOLO input size
        INPUT_WH_YOLO = 640
        blob = cv2.dnn.blobFromImage(input_image, 1/255, (INPUT_WH_YOLO, INPUT_WH_YOLO), swapRB=True, crop=False)
        self.yolo.setInput(blob)
        preds = self.yolo.forward()  # Detection or prediction from YOLO

        detections = preds[0]
        boxes = []
        confidences = []
        classes = []
        # Width and height of the frame (input_image)
        image_w, image_h = input_image.shape[:2]
        x_factor = image_w / INPUT_WH_YOLO
        y_factor = image_h / INPUT_WH_YOLO

        for i in range(len(detections)):
            row = detections[i]
            confidence = row[4]  # Confidence of detecting an object
            if confidence > 0.4:
                class_score = row[5:].max()  # Maximum probability of the object
                class_id = row[5:].argmax()  # Get the index position at which max probability occurs
                if class_score > 0.25:
                    cx, cy, w, h = row[0:4]
                    # Construct bounding box from four values: left top width height
                    left = int((cx - 0.5*w) * x_factor)
                    top = int((cy - 0.5*h) * y_factor)
                    width = int(w * x_factor)
                    height = int(h * y_factor)

                    box = np.array([left, top, width, height])

                    # Append values into the list
                    confidences.append(confidence)
                    boxes.append(box)
                    classes.append(class_id)

        # Clean the data
        boxes_np = np.array(boxes)
        confidences_np = np.array(confidences)

        # Apply Non-Maximum Suppression (NMS)
        nms_result = cv2.dnn.NMSBoxes(boxes_np.tolist(), confidences_np.flatten().tolist(), 0.25, 0.45)
        if len(nms_result) > 0:
            index = nms_result.flatten()
            # Draw bounding boxes and labels on the frame
            for ind in index:
                # Extract bounding boxes
                x, y, w, h = boxes_np[ind]
                bb_conf = int(confidences_np[ind] * 100)
                classes_id = classes[ind]
                class_name = self.labels[classes_id]
                colors = self.generate_colors(classes_id)

                text = f'{class_name}:{bb_conf}%'

                cv2.rectangle(frame, (x, y), (x+w, y+h), colors, 2)
                cv2.rectangle(frame, (x, y-30), (x+w, y), colors, -1)
                cv2.putText(frame, text, (x, y-10), cv2.FONT_HERSHEY_PLAIN, 0.7, (0, 0, 0), 1)
                        
        return frame

    def generate_colors(self, ID):
        np.random.seed(10)
        colors = np.random.randint(100, 255, size=(self.nc, 3)).tolist()
        return tuple(colors[ID])

def main():
    # Paths to the ONNX model and data YAML
    onnx_model = './Model/weights/best.onnx'
    data_yaml = 'data.yaml'

    yolo_detector = YOLO_pred(onnx_model, data_yaml)

    # Path to the input video file
    video_file = 'short.mp4'

    # Open the video file
    cap = cv2.VideoCapture(video_file)

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Perform predictions on the frame
        frame_with_predictions = yolo_detector.predictions(frame)

        # Display the frame with predictions
        cv2.imshow('YOLO Fire Detection', frame_with_predictions)
        
        # Press 'q' to exit
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Release resources
    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()


In [14]:
#detecting imageL
import cv2
import numpy as np
import yaml
from yaml.loader import SafeLoader

class YOLO_pred:
    def __init__(self, onnx_model, data_yaml):
        # Load YAML
        with open(data_yaml, mode='r') as f:
            data_yaml = yaml.load(f, Loader=SafeLoader)

        self.labels = data_yaml['names']
        self.nc = data_yaml['nc']

        # Load YOLO model
        self.yolo = cv2.dnn.readNetFromONNX(onnx_model)
        self.yolo.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
        self.yolo.setPreferableTarget(cv2.dnn.DNN_TARGET_CPU)

    def predictions(self, frame):
        # Get YOLO prediction from the frame
        row, col, d = frame.shape
        max_rc = max(row, col)
        input_image = np.zeros((max_rc, max_rc, 3), dtype=np.uint8)
        input_image[0:row, 0:col] = frame

        # Resize the image to fit YOLO input size
        INPUT_WH_YOLO = 640
        blob = cv2.dnn.blobFromImage(input_image, 1/255, (INPUT_WH_YOLO, INPUT_WH_YOLO), swapRB=True, crop=False)
        self.yolo.setInput(blob)
        preds = self.yolo.forward()  # Detection or prediction from YOLO

        detections = preds[0]
        boxes = []
        confidences = []
        classes = []
        # Width and height of the frame (input_image)
        image_w, image_h = input_image.shape[:2]
        x_factor = image_w / INPUT_WH_YOLO
        y_factor = image_h / INPUT_WH_YOLO

        for i in range(len(detections)):
            row = detections[i]
            confidence = row[4]  # Confidence of detecting an object
            if confidence > 0.4:
                class_score = row[5:].max()  # Maximum probability of the object
                class_id = row[5:].argmax()  # Get the index position at which max probability occurs
                if class_score > 0.25:
                    cx, cy, w, h = row[0:4]
                    # Construct bounding box from four values: left top width height
                    left = int((cx - 0.5*w) * x_factor)
                    top = int((cy - 0.5*h) * y_factor)
                    width = int(w * x_factor)
                    height = int(h * y_factor)

                    box = np.array([left, top, width, height])

                    # Append values into the list
                    confidences.append(confidence)
                    boxes.append(box)
                    classes.append(class_id)

        # Clean the data
        boxes_np = np.array(boxes)
        confidences_np = np.array(confidences)

        # Apply Non-Maximum Suppression (NMS)
        nms_result = cv2.dnn.NMSBoxes(boxes_np.tolist(), confidences_np.flatten().tolist(), 0.25, 0.45)
        if len(nms_result) > 0:
            index = nms_result.flatten()
            # Draw bounding boxes and labels on the frame
            for ind in index:
                # Extract bounding boxes
                x, y, w, h = boxes_np[ind]
                bb_conf = int(confidences_np[ind] * 100)
                classes_id = classes[ind]
                class_name = self.labels[classes_id]
                colors = self.generate_colors(classes_id)

                text = f'{class_name}:{bb_conf}%'

                cv2.rectangle(frame, (x, y), (x+w, y+h), colors, 2)
                cv2.rectangle(frame, (x, y-30), (x+w, y), colors, -1)
                cv2.putText(frame, text, (x, y-10), cv2.FONT_HERSHEY_PLAIN, 0.7, (0, 0, 0), 1)
                        
        return frame

    def generate_colors(self, ID):
        np.random.seed(10)
        colors = np.random.randint(100, 255, size=(self.nc, 3)).tolist()
        return tuple(colors[ID])

def main():
    # Paths to the ONNX model and data YAML
    onnx_model = './Model/weights/best.onnx'
    data_yaml = 'data.yaml'

    yolo_detector = YOLO_pred(onnx_model, data_yaml)

    # Path to the input image file
    image_file = 'design.jpg'

    # Read the input image
    frame = cv2.imread(image_file)

    # Perform predictions on the image
    frame_with_predictions = yolo_detector.predictions(frame)

    # Display the image with predictions
    cv2.imshow('YOLO Fire Detection', frame_with_predictions)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()


In [30]:
#for detecting image
import cv2
import numpy as np
import yaml
from yaml.loader import SafeLoader

class YOLO_pred:
    def __init__(self, onnx_model, data_yaml):
        # Load YAML
        with open(data_yaml, mode='r') as f:
            data_yaml = yaml.load(f, Loader=SafeLoader)

        self.labels = data_yaml['names']
        self.nc = data_yaml['nc']

        # Load YOLO model
        self.yolo = cv2.dnn.readNetFromONNX(onnx_model)
        self.yolo.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
        self.yolo.setPreferableTarget(cv2.dnn.DNN_TARGET_CPU)

    def predictions(self, frame):
        # Resize the frame to a reasonable size to ensure faster processing
        frame_resized = cv2.resize(frame, (640, 480))

        # Get YOLO prediction from the frame
        row, col, d = frame_resized.shape
        max_rc = max(row, col)
        input_image = np.zeros((max_rc, max_rc, 3), dtype=np.uint8)
        input_image[0:row, 0:col] = frame_resized

        # Resize the image to fit YOLO input size
        INPUT_WH_YOLO = 640
        blob = cv2.dnn.blobFromImage(input_image, 1/255, (INPUT_WH_YOLO, INPUT_WH_YOLO), swapRB=True, crop=False)
        self.yolo.setInput(blob)
        preds = self.yolo.forward()  # Detection or prediction from YOLO

        detections = preds[0]
        boxes = []
        confidences = []
        classes = []
        # Width and height of the frame (input_image)
        image_w, image_h = input_image.shape[:2]
        x_factor = image_w / INPUT_WH_YOLO
        y_factor = image_h / INPUT_WH_YOLO

        for i in range(len(detections)):
            row = detections[i]
            confidence = row[4]  # Confidence of detecting an object
            if confidence > 0.4:
                class_score = row[5:].max()  # Maximum probability of the object
                class_id = row[5:].argmax()  # Get the index position at which max probability occurs
                if class_score > 0.25:
                    cx, cy, w, h = row[0:4]
                    # Construct bounding box from four values: left top width height
                    left = int((cx - 0.5*w) * x_factor)
                    top = int((cy - 0.5*h) * y_factor)
                    width = int(w * x_factor)
                    height = int(h * y_factor)

                    box = np.array([left, top, width, height])

                    # Append values into the list
                    confidences.append(confidence)
                    boxes.append(box)
                    classes.append(class_id)

        # Clean the data
        boxes_np = np.array(boxes)
        confidences_np = np.array(confidences)

        # Apply Non-Maximum Suppression (NMS)
        nms_result = cv2.dnn.NMSBoxes(boxes_np.tolist(), confidences_np.flatten().tolist(), 0.25, 0.45)
        if len(nms_result) > 0:
            index = nms_result.flatten()
            # Draw bounding boxes and labels on the frame
            for ind in index:
                # Extract bounding boxes
                x, y, w, h = boxes_np[ind]
                bb_conf = int(confidences_np[ind] * 100)
                classes_id = classes[ind]
                class_name = self.labels[classes_id]
                colors = self.generate_colors(classes_id)

                text = f'{class_name}:{bb_conf}%'

                cv2.rectangle(frame_resized, (x, y), (x+w, y+h), colors, 2)
                cv2.rectangle(frame_resized, (x, y-30), (x+w, y), colors, -1)
                cv2.putText(frame_resized, text, (x, y-10), cv2.FONT_HERSHEY_PLAIN, 0.7, (0, 0, 0), 1)
                        
        return frame_resized

    def generate_colors(self, ID):
        np.random.seed(10)
        colors = np.random.randint(100, 255, size=(self.nc, 3)).tolist()
        return tuple(colors[ID])

def main():
    # Paths to the ONNX model and data YAML
    onnx_model = './Model/weights/best.onnx'
    data_yaml = 'data.yaml'

    yolo_detector = YOLO_pred(onnx_model, data_yaml)

    # Path to the input image file
    image_file = 'firee.jpg'

    # Read the input image
    frame = cv2.imread(image_file)

    # Perform predictions on the image
    frame_with_predictions = yolo_detector.predictions(frame)

    # Display the image with predictions
    cv2.imshow('YOLO Fire Detection', frame_with_predictions)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()
